#네이버 영화평 감성분석

In [4]:
!pip install konlpy > /dev/null

In [1]:
import pandas as pd
import numpy as np

In [2]:
#네이버 영화 리뷰 데이터 로 검색
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [3]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

## 1. 데이터 전처리

- 훈련데이터 셋

In [5]:
#중복 여부 확인
train_df.document.nunique()

146182

In [6]:
#중복데이터 배제
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [7]:
# Null 데이터 확인
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [10]:
# null 데이터 제거
train_df.dropna(how='any', inplace=True)
train_df.shape

(146182, 3)

In [11]:
# 긍정 부정 레이블의 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터 셋

In [12]:
test_df.document.nunique()

49157

In [14]:
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49158, 3)

In [17]:
test_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [18]:
test_df.dropna(how='any', inplace=True)
test_df.shape

(49157, 3)

In [19]:
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

## 텍스트 전처리
- train data set

In [20]:
# 한글과 공백 이외에는 제거
train_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')
train_df.head(2)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1


In [21]:
train_df['document'].replace('', np.nan, inplace=True)
train_df.document.isnull().sum()

391

In [22]:
train_df.dropna(how='any', inplace=True)


In [23]:
train_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')
train_df.head(2)
train_df['document'].replace('', np.nan, inplace=True)
train_df.document.isnull().sum()
train_df.shape

(145791, 3)

In [28]:
test_df['document'] = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')
test_df['document'].replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48995, 3)

## 3.한글처리

In [36]:
from konlpy.tag import Okt
okt = Okt()

In [29]:
from google.colab import files
uploaded = files.upload()
textfile = list(uploaded.keys())[0]

Saving 한국어불용어100.txt to 한국어불용어100.txt


In [33]:
df = pd.read_csv(textfile, sep='\s+', header=None)
df.head(2)

,0,1,2
0,이,VCP,0.018280
1,있,VA,0.011699


In [34]:
stopwords = set(df[0])
len(stopwords)

94

In [38]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정     '
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [39]:
okt.morphs(text, stem=True) #동사 원형으로 활용

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [40]:
' '.join(okt.morphs(text, stem=True)) #countervectorize에 넣기 위해 하나의 스트링으로 조절해야함

'교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정'

In [45]:
from tqdm.notebook import tqdm
str_train = []
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence.strip(), stem=True)
    temp_str = ' '.join(word for word in morphs if word not in stopwords)
    str_train.append(temp_str) 

  0%|          | 0/145791 [00:00<?, ?it/s]

In [46]:
str_train[:3]

['아 더빙 진짜 짜증나다 목소리',
 '흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다',
 '너 무재 밓었 다그 래서 보다 추천 다']

In [47]:
#테스트도 만들기
str_test = []
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence.strip(), stem=True)
    temp_str = ' '.join(word for word in morphs if word not in stopwords)
    str_test.append(temp_str) 

  0%|          | 0/48995 [00:00<?, ?it/s]

In [48]:
y_train = train_df.label.values    #Numpy Array
y_test = test_df.label.values

## 4. Feature 변환
- CountVectorizer, ngram_range(1,2)

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
# cvect = CountVectorizer(ngram_range=(1,2))
cvect = CountVectorizer()


In [53]:
cvect.fit(str_train)
X_train = cvect.transform(str_train)
X_test = cvect.transform(str_test)
X_train.shape, X_test.shape

((145791, 42092), (48995, 42092))

## 5.모델 학습/예측/평가

In [54]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [57]:
lr = LogisticRegression(max_iter=500)
%time lr.fit(X_train, y_train)

CPU times: user 18.1 s, sys: 19.6 s, total: 37.7 s
Wall time: 19.4 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
pred = lr.predict(X_test)
accuracy_score(y_test, pred)

0.8259006021022554

## 6.실제 테스트


In [75]:
review1 = "처음에 황정민 실제 연기나 시상식 나오는 거부터 약간 반칙임..ㅎ 진짜에서 픽션으로 넘어갈 때도 다 믿어짐 황정민 황정민 하는거 이유가 있다"
review2 =" 다들 알바인가요. 정말 올드한 영화였습니다."


In [61]:
review2.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '') # 돌아가긴 하지만 변하지 않음

' 다들 알바인가요. 정말 올드한 영화였습니다.'

In [76]:
import re

In [90]:
review1= re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review1)
review2= re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review2)

In [93]:
review1

'처음에 황정민 실제 연기나 시상식 나오는 거부터 약간 반칙임ㅎ 진짜에서 픽션으로 넘어갈 때도 다 믿어짐 황정민 황정민 하는거 이유가 있다'

In [98]:
reviews = [review1, review2]
review_list = []
for review in reviews:
    review= re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review)
    morphs = okt.morphs(review.strip(), stem=True)
    temp_str = ' '.join(word for word in morphs if word not in stopwords)
    review_list.append(temp_str)


In [99]:
review_list

['처음 에 황정민 실제 연기 시상식 나오다 거 부터 약간 반칙 임 ㅎ 진짜 에서 픽션 으로 넘어가다 도 다 믿어지다 황정민 황정민 하다 이유 있다',
 '다 들다 알바 인가요 정말 올드 영화 이다']

In [100]:
review_cv =  cvect.transform(review_list)
pred = lr.predict(review_cv)

In [101]:
pred

array([1, 0])

In [ ]:
#모델 저정 joblib